In [11]:
import numpy as np
import pickle as pickle
import scipy.interpolate as interp
import scipy.signal as signal
import scipy.optimize as opti
import scipy, sys, time
import matplotlib.pyplot as plt
from matplotlib.mlab import psd
from tqdm import tqdm
import os.path

In [12]:
def find_ind(pos):
    if np.abs(pos) <= 25e-6:
        ind = 0
    elif np.abs(pos) > 25e-6:
        if np.abs(pos) <= 75e-6:
            if pos > 0:
                ind = 1.0
            elif pos < 0:
                ind = -1.0
        elif np.abs(pos) > 75e-6:
            if np.abs(pos) <= 125e-6:
                if pos > 0:
                    ind = 2.0
                if pos < 0:
                    ind = -2.0
            elif np.abs(pos) > 125e-6:
                if np.abs(pos) <= 175e-6:
                    if pos > 0:
                        ind = 3.0
                    if pos < 0:
                        ind = -3.0
                elif np.abs(pos) > 175e-6:
                    if pos > 0:
                        ind = 4.0
                    if pos < 0:
                        ind = -4.0

    newpos = pos - ind * 50e-6

    return ind, newpos




def dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def dist_p_arrp(p1, xs, ys, zs):
    xnew = (xs - p1[0])**2
    ynew = (ys - p1[1])**2
    znew = (zs - p1[2])**2
    return np.sqrt(np.add.outer(np.add.outer(xnew, ynew), znew))

In [18]:
def simulation_paralellisatzion(sep):
    start = time.time()

    # bead radius [µm] (certain it shouldn't be diameter)
    rbead = 2.4e-6 #float(sys.argv[1])
    # separation attractor [µm] - to surface of the bead
    #sep = 15e-6 #float(sys.argv[2])
    # difference in height of attractor and bead [µm]
    height = 0 #float(sys.argv[3])
    # load the density profile of the attractor (took the one AlexR and Chas have produced)
    rhopath = '/home/analysis_user/New_trap_code/SensitivityFramework/results/rho_arr.p'
    rho, xx, yy, zz = pickle.load(open(rhopath, 'rb'), encoding="latin1") # first change, encoding to latin1 instead of ASCII
    print("Density Loaded.")
    sys.stdout.flush()
    
    rbead = 3.8e-6 #float(sys.argv[1])
    height = 0 #float(sys.argv[3])

    xx = np.array(xx) # maxe array for x
    yy = np.array(yy) # make array for y
    zz = np.array(zz) # make array for z

    xinds = np.abs(xx) <= 25.0e-6 #
    yinds = np.abs(yy) <= 80.0e-6 #
    zinds = np.abs(zz) <= 5.0e-6 #

    xx2 = xx[xinds] #
    yy2 = yy[yinds] #
    zz2 = zz[zinds] #
    rho2 = rho[xinds,:,:][:,yinds,:] #


    # produce empty arrays 

    xzeros = np.zeros(len(xx))
    yzeros = np.zeros(len(yy))
    zzeros = np.zeros(len(zz))

    xzeros2 = np.zeros(len(xx2))
    yzeros2 = np.zeros(len(yy2))
    zzeros2 = np.zeros(len(zz2))

    dx = np.abs(xx[1] - xx[0])
    dy = np.abs(yy[1] - yy[0])
    dz = np.abs(zz[1] - zz[0])

    cell_volume = dx * dy * dz
    m = rho * cell_volume
    m2 = rho2 * cell_volume
    
    G = 6.67e-11     # m^3 / (kg s^2)
    rhobead = 1550.

    travel = 500e-6 # travel
    cent = 0.0e-6 # where to start
    Npoints = 1001 # how many points for each direction
    beadposvec = np.linspace(cent - travel*0.5, cent + travel*0.5, Npoints)
    beadposvec2 = np.linspace(-1.0 * travel, travel, 2*(Npoints-1) + 1)

    lambdas = np.logspace(-6.3, -3, 100)
    lambdas = lambdas[::-1]
    
    # where to put the results and how to name them

    respath = '/home/analysis_user/New_trap_code/SensitivityFramework/results/simulation/large_bead/'
    respath = respath + 'rbead_' + str(rbead)
    respath = respath + '_sep_%4.1e' %(sep)
    respath = respath + '_height_' + str(height)
    respath = respath + '.p'
    results_dic = {}
    results_dic['order'] = 'Rbead, Sep, Height, Yuklambda'

    if(os.path.exists(respath)==False):

        finger_inds = np.array([-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0])


        results_dic[rbead] = {}
        results_dic[rbead][sep] = {}
        results_dic[rbead][sep][height] = {}


        Gterm = 2. * rbead**3

        GforcecurveX = []
        GforcecurveY = []
        GforcecurveZ = []

        for ind, xpos in enumerate(beadposvec2):
            beadpos = [xpos, sep+rbead, height]

            s = dist_p_arrp(beadpos, xx2, yy2, zz2) - rbead

            # These are used to compute projections and thus need to maintain sign
            xsep = dist_p_arrp([xpos, 0, 0], xx2, yzeros2, zzeros2)
            xind = np.argmin(np.abs(xx2 - xpos))
            xsep[:xind,:,:] *= -1.0

            # We a priori know that all yseps should be negative
            ysep = dist_p_arrp([0, sep+rbead, 0], xzeros2, yy2, zzeros2)
            ysep *= -1.0

            zsep = dist_p_arrp([0, 0, height], xzeros2, yzeros2, zz2)
            zind = np.argmin(np.abs(zz2 - height))
            zsep[:,:,:zind] *= -1.0
            #print(zsep[0,0,0])    


            xprojection = xsep / (s + rbead)
            yprojection = ysep / (s + rbead)
            zprojection = zsep / (s + rbead)

            prefac = ((2. * G * m2 * rhobead * np.pi) / (3. * (rbead + s)**2))

            xtotforce = np.sum(prefac * Gterm * xprojection)
            ytotforce = np.sum(prefac * Gterm * yprojection)
            ztotforce = np.sum(prefac * Gterm * zprojection)

            # SWAP X AND Y AXES TO MATCH DATA AXES
            GforcecurveX.append(ytotforce)
            GforcecurveY.append(xtotforce)
            GforcecurveZ.append(ztotforce)

        GforcecurveX = np.array(GforcecurveX)
        GforcecurveY = np.array(GforcecurveY)
        GforcecurveZ = np.array(GforcecurveZ)
        #plt.plot(GforcecurveY)
        GX = interp.interp1d(beadposvec2, GforcecurveX, kind='cubic')
        GY = interp.interp1d(beadposvec2, GforcecurveY, kind='cubic')
        GZ = interp.interp1d(beadposvec2, GforcecurveZ, kind='cubic')
        newGX = np.zeros(len(beadposvec))
        newGY = np.zeros(len(beadposvec))
        newGZ = np.zeros(len(beadposvec))

        for ind, pos in enumerate(beadposvec):
            if np.abs(pos) - 225e-6 > 0:
                newGX[ind] = 0.0
                newGY[ind] = 0.0
                newGZ[ind] = 0.0
                continue

            finger_ind, newpos = find_ind(pos)
            width = 50.0e-6

            newGX[ind] = np.sum(GX(newpos + (finger_inds+finger_ind) * width))
            newGY[ind] = np.sum(GY(newpos + (finger_inds+finger_ind) * width))
            newGZ[ind] = np.sum(GZ(newpos + (finger_inds+finger_ind) * width))  

        print('Computed normal grav')
        sys.stdout.flush()

        for yukind, yuklambda in enumerate(tqdm(lambdas)):

            func = np.exp(-2. * rbead / yuklambda) * (1. + rbead / yuklambda) + rbead / yuklambda - 1.

            yukforcecurveX = []
            yukforcecurveY = []
            yukforcecurveZ = []
            for ind, xpos in enumerate(beadposvec2):

                beadpos = [xpos, sep+rbead, height]

                s = dist_p_arrp(beadpos, xx2, yy2, zz2) - rbead
                # These are used to compute projections and thus need to maintain sign
                xsep = dist_p_arrp([xpos, 0, 0], xx2, yzeros2, zzeros2)
                xind = np.argmin(np.abs(xx2 - xpos))
                xsep[:xind,:,:] *= -1.0

                # We a priori know that all yseps should be negative
                ysep = dist_p_arrp([0, sep+rbead, 0], xzeros2, yy2, zzeros2)
                ysep *= -1.0

                zsep = dist_p_arrp([0, 0, height], xzeros2, yzeros2, zz2)
                zind = np.argmin(np.abs(zz2 - height))
                zsep[:,:,:zind] *= -1.0

                xprojection = xsep / (s + rbead)
                yprojection = ysep / (s + rbead)
                zprojection = zsep / (s + rbead)

                prefac = ((2. * G * m2 * rhobead * np.pi) / (3. * (rbead + s)**2))

                yukterm = 3 * yuklambda**2 * (rbead + s + yuklambda) * func * np.exp( - s / yuklambda )
                #print(-s/yuklambda)
                xtotforce = np.sum(prefac * yukterm * xprojection)
                ytotforce = np.sum(prefac * yukterm * yprojection)
                ztotforce = np.sum(prefac * yukterm * zprojection)


                # SWAP X AND Y AXES TO MATCH DATA AXES
                yukforcecurveX.append(ytotforce) 
                yukforcecurveY.append(xtotforce)
                yukforcecurveZ.append(ztotforce)

            yukforcecurveX = np.array(yukforcecurveX)
            yukforcecurveY = np.array(yukforcecurveY)
            yukforcecurveZ = np.array(yukforcecurveZ)

            yukX = interp.interp1d(beadposvec2, yukforcecurveX, kind='cubic')
            yukY = interp.interp1d(beadposvec2, yukforcecurveY, kind='cubic')
            yukZ = interp.interp1d(beadposvec2, yukforcecurveZ, kind='cubic')

            newyukX = np.zeros(len(beadposvec))
            newyukY = np.zeros(len(beadposvec))
            newyukZ = np.zeros(len(beadposvec))

            for ind, pos in enumerate(beadposvec):
                if np.abs(pos) - 225e-6 > 0:
                    newyukX[ind] = 0.0
                    newyukY[ind] = 0.0
                    newyukZ[ind] = 0.0
                    continue

                finger_ind, newpos = find_ind(pos)
                width = 50.0e-6

                newyukX[ind] = np.sum(yukX(newpos + (finger_inds+finger_ind) * width))
                newyukY[ind] = np.sum(yukY(newpos + (finger_inds+finger_ind) * width))
                newyukZ[ind] = np.sum(yukZ(newpos + (finger_inds+finger_ind) * width))
                #print(newyukX[ind],newyukY[ind],newyukZ[ind])


            results_dic[rbead][sep][height][yuklambda] = \
                        (newGX, newGY, newGZ, newyukX, newyukY, newyukZ)





        print("100! Done!")
        end = time.time()
        print("Time: %d" %(end-start))
        results_dic['posvec'] = beadposvec
        try:
            pickle.dump(results_dic, open(respath, 'wb') )
            print("Saved: ", respath)
        except:
            print("Save didn't work!")
    else:print("File already exists, skipping")
    return results_dic

In [19]:
from joblib import Parallel, delayed
sep_list = np.arange(1.0e-6,20.0e-6,1.0e-6)
sep_result = Parallel(n_jobs=8)(delayed(simulation_paralellisatzion)(sep) for sep in sep_list)


KeyboardInterrupt: 

In [24]:
def simulation_paralellisatzion(height):
    start = time.time()
    sep_list = np.arange(0.0e-6,50e-6,0.5e-6)
    for sep in sep_list:
        # bead radius [µm] (certain it shouldn't be diameter)
        rbead = 3.8e-06 #float(sys.argv[1])
        # separation attractor [µm] - to surface of the bead
        #sep = 15e-6 #float(sys.argv[2])
        # difference in height of attractor and bead [µm]
        height = height #float(sys.argv[3])
        # load the density profile of the attractor (took the one AlexR and Chas have produced)
        rhopath = '/home/analysis_user/New_trap_code/SensitivityFramework/results/rho_arr.p'
        rho, xx, yy, zz = pickle.load(open(rhopath, 'rb'), encoding="latin1") # first change, encoding to latin1 instead of ASCII
        print("Density Loaded.")
        sys.stdout.flush()

  

        xx = np.array(xx) # maxe array for x
        yy = np.array(yy) # make array for y
        zz = np.array(zz) # make array for z

        xinds = np.abs(xx) <= 25.0e-6 #
        yinds = np.abs(yy) <= 80.0e-6 #
        zinds = np.abs(zz) <= 5.0e-6 #

        xx2 = xx[xinds] #
        yy2 = yy[yinds] #
        zz2 = zz[zinds] #
        rho2 = rho[xinds,:,:][:,yinds,:] #


        # produce empty arrays 

        xzeros = np.zeros(len(xx))
        yzeros = np.zeros(len(yy))
        zzeros = np.zeros(len(zz))

        xzeros2 = np.zeros(len(xx2))
        yzeros2 = np.zeros(len(yy2))
        zzeros2 = np.zeros(len(zz2))

        dx = np.abs(xx[1] - xx[0])
        dy = np.abs(yy[1] - yy[0])
        dz = np.abs(zz[1] - zz[0])

        cell_volume = dx * dy * dz
        m = rho * cell_volume
        m2 = rho2 * cell_volume

        G = 6.67e-11     # m^3 / (kg s^2)
        rhobead = 1550.

        travel = 500e-6 # travel
        cent = 0.0e-6 # where to start
        Npoints = 1001 # how many points for each direction
        beadposvec = np.linspace(cent - travel*0.5, cent + travel*0.5, Npoints)
        beadposvec2 = np.linspace(-1.0 * travel, travel, 2*(Npoints-1) + 1)

        lambdas = np.logspace(-6.3, -3, 100)
        lambdas = lambdas[::-1]

        # where to put the results and how to name them

        respath = '/home/analysis_user/New_trap_code/SensitivityFramework/results/simulation/bead_%4.1e/' %rbead
        respath = respath + 'rbead_' + str(rbead)
        respath = respath + '_sep_%4.1e' %(sep)
        respath = respath + '_height_%4.1e' %(height)
        respath = respath + '.p'
        print(respath)
        results_dic = {}
        results_dic['order'] = 'Rbead, Sep, Height, Yuklambda'

        if(os.path.exists(respath)==False):

            finger_inds = np.array([-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0])


            results_dic[rbead] = {}
            results_dic[rbead][sep] = {}
            results_dic[rbead][sep][height] = {}


            Gterm = 2. * rbead**3

            GforcecurveX = []
            GforcecurveY = []
            GforcecurveZ = []

            for ind, xpos in enumerate(beadposvec2):
                beadpos = [xpos, sep+rbead, height]

                s = dist_p_arrp(beadpos, xx2, yy2, zz2) - rbead

                # These are used to compute projections and thus need to maintain sign
                xsep = dist_p_arrp([xpos, 0, 0], xx2, yzeros2, zzeros2)
                xind = np.argmin(np.abs(xx2 - xpos))
                xsep[:xind,:,:] *= -1.0

                # We a priori know that all yseps should be negative
                ysep = dist_p_arrp([0, sep+rbead, 0], xzeros2, yy2, zzeros2)
                ysep *= -1.0

                zsep = dist_p_arrp([0, 0, height], xzeros2, yzeros2, zz2)
                zind = np.argmin(np.abs(zz2 - height))
                zsep[:,:,:zind] *= -1.0
                #print(zsep[0,0,0])    


                xprojection = xsep / (s + rbead)
                yprojection = ysep / (s + rbead)
                zprojection = zsep / (s + rbead)

                prefac = ((2. * G * m2 * rhobead * np.pi) / (3. * (rbead + s)**2))

                xtotforce = np.sum(prefac * Gterm * xprojection)
                ytotforce = np.sum(prefac * Gterm * yprojection)
                ztotforce = np.sum(prefac * Gterm * zprojection)

                # SWAP X AND Y AXES TO MATCH DATA AXES
                GforcecurveX.append(ytotforce)
                GforcecurveY.append(xtotforce)
                GforcecurveZ.append(ztotforce)

            GforcecurveX = np.array(GforcecurveX)
            GforcecurveY = np.array(GforcecurveY)
            GforcecurveZ = np.array(GforcecurveZ)
            #plt.plot(GforcecurveY)
            GX = interp.interp1d(beadposvec2, GforcecurveX, kind='cubic')
            GY = interp.interp1d(beadposvec2, GforcecurveY, kind='cubic')
            GZ = interp.interp1d(beadposvec2, GforcecurveZ, kind='cubic')
            newGX = np.zeros(len(beadposvec))
            newGY = np.zeros(len(beadposvec))
            newGZ = np.zeros(len(beadposvec))

            for ind, pos in enumerate(beadposvec):
                if np.abs(pos) - 225e-6 > 0:
                    newGX[ind] = 0.0
                    newGY[ind] = 0.0
                    newGZ[ind] = 0.0
                    continue

                finger_ind, newpos = find_ind(pos)
                width = 50.0e-6

                newGX[ind] = np.sum(GX(newpos + (finger_inds+finger_ind) * width))
                newGY[ind] = np.sum(GY(newpos + (finger_inds+finger_ind) * width))
                newGZ[ind] = np.sum(GZ(newpos + (finger_inds+finger_ind) * width))  

            print('Computed normal grav')
            sys.stdout.flush()

            for yukind, yuklambda in enumerate(tqdm(lambdas)):

                func = np.exp(-2. * rbead / yuklambda) * (1. + rbead / yuklambda) + rbead / yuklambda - 1.

                yukforcecurveX = []
                yukforcecurveY = []
                yukforcecurveZ = []
                for ind, xpos in enumerate(beadposvec2):

                    beadpos = [xpos, sep+rbead, height]

                    s = dist_p_arrp(beadpos, xx2, yy2, zz2) - rbead
                    # These are used to compute projections and thus need to maintain sign
                    xsep = dist_p_arrp([xpos, 0, 0], xx2, yzeros2, zzeros2)
                    xind = np.argmin(np.abs(xx2 - xpos))
                    xsep[:xind,:,:] *= -1.0

                    # We a priori know that all yseps should be negative
                    ysep = dist_p_arrp([0, sep+rbead, 0], xzeros2, yy2, zzeros2)
                    ysep *= -1.0

                    zsep = dist_p_arrp([0, 0, height], xzeros2, yzeros2, zz2)
                    zind = np.argmin(np.abs(zz2 - height))
                    zsep[:,:,:zind] *= -1.0

                    xprojection = xsep / (s + rbead)
                    yprojection = ysep / (s + rbead)
                    zprojection = zsep / (s + rbead)

                    prefac = ((2. * G * m2 * rhobead * np.pi) / (3. * (rbead + s)**2))

                    yukterm = 3 * yuklambda**2 * (rbead + s + yuklambda) * func * np.exp( - s / yuklambda )
                    #print(-s/yuklambda)
                    xtotforce = np.sum(prefac * yukterm * xprojection)
                    ytotforce = np.sum(prefac * yukterm * yprojection)
                    ztotforce = np.sum(prefac * yukterm * zprojection)


                    # SWAP X AND Y AXES TO MATCH DATA AXES
                    yukforcecurveX.append(ytotforce) 
                    yukforcecurveY.append(xtotforce)
                    yukforcecurveZ.append(ztotforce)

                yukforcecurveX = np.array(yukforcecurveX)
                yukforcecurveY = np.array(yukforcecurveY)
                yukforcecurveZ = np.array(yukforcecurveZ)

                yukX = interp.interp1d(beadposvec2, yukforcecurveX, kind='cubic')
                yukY = interp.interp1d(beadposvec2, yukforcecurveY, kind='cubic')
                yukZ = interp.interp1d(beadposvec2, yukforcecurveZ, kind='cubic')

                newyukX = np.zeros(len(beadposvec))
                newyukY = np.zeros(len(beadposvec))
                newyukZ = np.zeros(len(beadposvec))

                for ind, pos in enumerate(beadposvec):
                    if np.abs(pos) - 225e-6 > 0:
                        newyukX[ind] = 0.0
                        newyukY[ind] = 0.0
                        newyukZ[ind] = 0.0
                        continue

                    finger_ind, newpos = find_ind(pos)
                    width = 50.0e-6

                    newyukX[ind] = np.sum(yukX(newpos + (finger_inds+finger_ind) * width))
                    newyukY[ind] = np.sum(yukY(newpos + (finger_inds+finger_ind) * width))
                    newyukZ[ind] = np.sum(yukZ(newpos + (finger_inds+finger_ind) * width))
                    #print(newyukX[ind],newyukY[ind],newyukZ[ind])


                results_dic[rbead][sep][height][yuklambda] = \
                            (newGX, newGY, newGZ, newyukX, newyukY, newyukZ)





            print("100! Done!")
            end = time.time()
            print("Time: %d" %(end-start))
            results_dic['posvec'] = beadposvec
            try:
                pickle.dump(results_dic, open(respath, 'wb') )
                print("Saved: ", respath)
            except:
                print("Save didn't work!")
        else:print("File already exists, skipping")
    return results_dic

In [ ]:
from joblib import Parallel, delayed
height_list = np.arange(-15.0e-6,15.0e-6,0.5e-6)
height_result = Parallel(n_jobs=8)(delayed(simulation_paralellisatzion)(height) for height in tqdm(height_list))




  0%|          | 0/61 [00:00<?, ?it/s]

 26%|██▌       | 16/61 [00:46<02:10,  2.90s/it]

 28%|██▊       | 17/61 [00:46<01:32,  2.11s/it]

 30%|██▉       | 18/61 [00:47<01:11,  1.66s/it]

 31%|███       | 19/61 [00:49<01:10,  1.69s/it]

 33%|███▎      | 20/61 [00:49<00:50,  1.22s/it]

 34%|███▍      | 21/61 [00:52<01:16,  1.90s/it]

 36%|███▌      | 22/61 [00:54<01:09,  1.77s/it]

 38%|███▊      | 23/61 [00:56<01:10,  1.85s/it]

 39%|███▉      | 24/61 [01:32<07:25, 12.05s/it]

 41%|████      | 25/61 [01:34<05:34,  9.30s/it]

 43%|████▎     | 26/61 [01:35<03:54,  6.69s/it]

 44%|████▍     | 27/61 [01:36<02:51,  5.03s/it]

 46%|████▌     | 28/61 [01:38<02:15,  4.10s/it]

 48%|████▊     | 29/61 [01:39<01:44,  3.26s/it]

 49%|████▉     | 30/61 [01:43<01:40,  3.24s/it]

 51%|█████     | 31/61 [01:45<01:31,  3.06s/it]

 52%|█████▏    | 32/61 [02:19<05:58, 12.38s/it]

 54%|█████▍    | 33/61 [02:21<04:16,  9.15s/it]

 56%|█████▌    | 34/61 [02:21<02:56,  6.52s/it]

 57%|█████▋    | 35/61 [02:

In [ ]:
print(test)